In [ ]:
!pip install textblob 'gensim==4.2.0' 'keras-nlp' swifter

In [1]:
import multiprocessing
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout
import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras import preprocessing
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import Model, Input
import swifter
import keras_nlp
import numpy as np
import re
import random
import os
import pandas as pd
import gensim
import warnings
import nltk

embedding_dim = 256
rnn_units = 1024
epochs=5
buffer_size = 2000
# Batch size
batch_size = 256
fraction_corpus = 0.02
BATCH = True
TRACE = False

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config) 
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

2024-11-03 16:04:36.024823: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-03 16:04:36.068986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using TensorFlow backend


/home/alyydi/pluralsight/NLPCert/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-03 16:04:40.156305: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-03 16:04:40.156337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: VDL900341
2024-11-03 16:04:40.156341: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: VDL900341
2024-11-03 16:04:40.156444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2024-11-03 16:04:40.156471: I tensorflow/compiler/xla/stream_executor/cuda/c

In [2]:
def preprocess_text(text, should_join=True):
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"(\@\d+)", r" ", text) # remove twitter handle
    text = re.sub(r"(https\:\/\/t.co\/\w+)", r" ", text) # remove end twtter url
    text = re.sub(r"([.,!?])", r" ", text) # remove punctuation
    text = re.sub(r"[^a-zA-Z.,&!?]+", r" ", text) # only normal characters
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [3]:

path = './twitter.csv'
twitter = pd.read_csv(path, header=0)
twitter

FileNotFoundError: [Errno 2] No such file or directory: './twitter.csv'